In [1]:
#import library
from libsbml import readSBML
from cobra import io
from cobra.core import Metabolite
from Functions import find_average

#import sbml file
model = io.sbml.create_cobra_model_from_sbml_file("/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_2_3.xml")

#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})

In [2]:
import pandas as pd
biomass = pd.DataFrame(data={"":["sSUCROSE_b","GLC_c","FRU_c","Starch_b","Cellulose_b","PALMITATE_c",
                                 "L_PHOSPHATIDATE_p","PHOSPHATIDYL_CHOLINE_r",
                                 "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","DIACYLGLYCEROL_p","Protein_b","sMAL_b",
                                 "sCIT_b","sFUM_b","ARG_c","HIS_c","LYS_c","sASP_b",
                                 "sGLU_b","sSER_b","THR_c","ASN_c","sGLN_b","CYS_c",
                                 "GLY_c","PRO_c","sALA_b","VAL_c","ILE_c","LEU_c",
                                 "MET_c","PHE_c","TYR_c","TRP_c","sGABA_b"],
                             "leaf":[0.0]*35,"stem":[0.0]*35,"root":[0.0]*35,"seed":[0.0]*35,},dtype="float64")
biomass = biomass.set_index("")

### Leaf

Data extracted from Allwood et al 2015 Figure 1, was used to identify age of mature photosynthetic cells

<img src="references/Screenshot_20190710_102223.png">


Given that the mesophyll-cell-number-per-gram-FW stabilizes 40+ mm away from leaf base, cells are fully expanded in this region of the leaf

In [3]:
df_age_dist = pd.DataFrame(data={"1- distance from leaf base (mm)":[4.90932322908728,9.81901468871633,
                                                                 19.8310558274325,24.8076179534483,
                                                                 29.9084947103683,34.8212793065482,
                                                                 39.6748524316104,44.7119516586945,
                                                                 49.8770478221005,54.6650759107262,
                                                                 59.7643324532624,64.8049666935475,
                                                                 69.7206971340617,74.6925459091428,
                                                                 79.6084236418737,84.6469957911249,],
                                "2- cell age (h)":[6.79783038564788,14.7759133038013,27.0455022480475,
                                                32.0926173458679,37.3297394032456,43.3220286557008,
                                                47.3280823658076,53.51037857782,57.5186416711775,
                                                63.6046087734609,69.8819084652519,73.7947262021807,
                                                77.8957833920661,85.9688697899981,89.9753653767551,
                                                95.2120455574826]})
def cellAgefromDistancefromLeafBase(x,m,c):
    y = (m*x)+c
    return y

from scipy.optimize import curve_fit
popt_cellAge, pcov_cellAge = curve_fit(cellAgefromDistancefromLeafBase, \
                                       df_age_dist["1- distance from leaf base (mm)"], \
                                       df_age_dist["2- cell age (h)"],method="lm")

xlist = list()
ylist = list()
for i in range(5,95,10):
    i=float(i)
    xlist.append(i)
    ylist.append(cellAgefromDistancefromLeafBase(i,*popt_cellAge))

import matplotlib.pyplot as py
py.plot(df_age_dist["1- distance from leaf base (mm)"],df_age_dist["2- cell age (h)"],"o",label="data")
py.plot(xlist,ylist,"-",label="fitting")
py.xlabel("distance from leaf base (mm)")
py.ylabel("cell age (h)")
py.legend()
py.show()

<Figure size 640x480 with 1 Axes>

Extracting data from Allwood et al 2015 Figure 3

<img src="references/Screenshot_20190710_111129.png">

In [4]:
df_content_1 = pd.DataFrame(data={"1- cell age (h)":[10.7692307692308,21.7751479289941,31.8343195266272,
                                                     41.8343195266272,51.8343195266272,61.7751479289941,
                                                     71.8343195266272,81.8343195266272,91.8934911242603],
                                "2- soluble CHO (mg/gFW)":[11,9.78947368421052,5.52631578947369,3.47368421052631,
                                                        3.10526315789473,2.99999999999999,4.31578947368421,
                                                        5.94736842105263,6.78947368421052],
                                 "3- insoluble CHO (mg/gFW)":[3.8515254521801,5.61828024675814,6.5047631037811,
                                                           5.37722942632925,4.76016618406144,4.02962776448865,
                                                           3.52603969952579,3.64673297242856,3.6254144110118],
                                 "4- protein concentration (mg/gFW)":[78.2456140350878,34.7368421052633,
                                                                   38.9473684210529,45.6140350877195,
                                                                   49.1228070175441,57.5438596491231,
                                                                   59.6491228070178,60.3508771929828,
                                                                   70.526315789474],
                                 "5- amino acid (umol/gFW)":[62.8169014084507,27.8873239436621,23.6619718309861,
                                                          22.5352112676059,21.1267605633804,23.6619718309861,
                                                          28.4507042253523,30.985915492958,40.8450704225355]})
df_content_1

,1- cell age (h),2- soluble CHO (mg/gFW),3- insoluble CHO (mg/gFW),4- protein concentration (mg/gFW),5- amino acid (umol/gFW)
0,10.769231,11.000000,3.851525,78.245614,62.816901
1,21.775148,9.789474,5.618280,34.736842,27.887324
2,31.834320,5.526316,6.504763,38.947368,23.661972
3,41.834320,3.473684,5.377229,45.614035,22.535211
4,51.834320,3.105263,4.760166,49.122807,21.126761
5,61.775148,3.000000,4.029628,57.543860,23.661972
6,71.834320,4.315789,3.526040,59.649123,28.450704
7,81.834320,5.947368,3.646733,60.350877,30.985915
8,91.893491,6.789474,3.625414,70.526316,40.845070


FW/DW ratio was estimated from Hu et al 2006

<img src="references/Screenshot_20190711_085710.png" height=50% width=50%>

In [5]:
df_FW_DW = pd.DataFrame(data={"Blade FW(g)":[0.2609523809523799,0.28761904761904644],
                              "Blade DW(g)":[0.046863468634686364,0.05409594095940953]})
df_FW_DW["FW/DW"]=df_FW_DW["Blade FW(g)"]/df_FW_DW["Blade DW(g)"]
print df_FW_DW
print("Average FW/DW = "+str(sum(df_FW_DW["FW/DW"])/len(df_FW_DW["FW/DW"])))

   Blade DW(g)  Blade FW(g)     FW/DW
0     0.046863     0.260952  5.568354
1     0.054096     0.287619  5.316832
Average FW/DW = 5.44259314107


In [6]:
list1 = list(df_content_1["2- soluble CHO (mg/gFW)"]*5.44259314107)
list2 = list(df_content_1["3- insoluble CHO (mg/gFW)"]*5.44259314107)
list3 = list(df_content_1["4- protein concentration (mg/gFW)"]*5.44259314107)
list4 = list(df_content_1["5- amino acid (umol/gFW)"]*5.44259314107)

df_content_1_std = pd.DataFrame(data={"1- cell age (h)":df_content_1["1- cell age (h)"],
                                      "2- soluble CHO (mg/gDW)":list1,
                                      "3- insoluble CHO (mg/gDW)":list2,
                                      "4- protein concentration (mg/gDW)":list3,
                                      "5- amino acid (umol/gDW)":list4})
df_content_1_std

,1- cell age (h),2- soluble CHO (mg/gDW),3- insoluble CHO (mg/gDW),4- protein concentration (mg/gDW),5- amino acid (umol/gDW)
0,10.769231,59.868525,20.962286,425.859042,341.886837
1,21.775148,53.280122,30.578014,189.058499,151.779358
2,31.834320,30.077488,35.402779,211.974680,128.782486
3,41.834320,18.905850,29.266072,248.258635,122.649986
4,51.834320,16.900684,25.907648,267.355453,114.984362
5,61.775148,16.327779,21.931624,313.187816,128.782486
6,71.834320,23.489086,19.190799,324.645907,154.845608
7,81.834320,32.369107,19.847684,328.465270,168.643731
8,91.893491,36.952343,19.731656,383.846043,222.303100


Extracting data from Shi et al 2016 Figure 4

<img src="references/Screenshot_20190710_115400.png">

In [7]:
time = 30*24
print("30 DPA in hours = "+str(time))
df_content_2 = pd.DataFrame(data={"DPA":[15,15,15,15,20,20,20,20,25,25,25,25,30,30,30,30],
                                  "sucrose content (mg/gDW)":[10.4865764650646,14.2319347816808,
                                                              47.4902898117717,57.977293098297,
                                                              19.7754919117333,19.1762345810748,
                                                              53.3339024286141,62.7722053864868,
                                                              84.9447266208545,78.3537496265312,
                                                              23.970720047804,14.2332152460626,
                                                              12.2864825643433,11.3875965683554,
                                                              2.99799393913526,1.35003627982414]})
df_content_2["DPA"]=df_content_2["DPA"]*24
df_content_2

30 DPA in hours = 720


,DPA,sucrose content (mg/gDW)
0,360,10.486576
1,360,14.231935
2,360,47.490290
3,360,57.977293
4,480,19.775492
5,480,19.176235
6,480,53.333902
7,480,62.772205
8,600,84.944727
9,600,78.353750


### Summary

- Unable to combining Allwood et al 2015 data with Shi et al 2016 because 1) the tissue have different age 2) total soluble carbohydrate content reported in the first study is drastically lower than sucrose content in the latter study

Assumptions:

- all soluble CHO were assumed to be sucrose
- all insoluble CHO was assumed to be starch
- protein composition data is not available and so the default core mdoel composition was used
- amino acid composition data is not available and so an even distribution was assumed

In [8]:


df_content_std_1 = pd.DataFrame(data={"1- sucrose (mg/gDW)":[df_content_1_std["2- soluble CHO (mg/gDW)"][8]],
                                    "2- insoluble CHO (mg/gDW)":[df_content_1_std["3- insoluble CHO (mg/gDW)"][8]],
                                    "3- protein concentration (mg/gDW)":[df_content_1_std["4- protein concentration (mg/gDW)"][8]],
                                    "4- amino acid (umol/gDW)":[df_content_1_std["5- amino acid (umol/gDW)"][8]]})
df_content_std_1


,1- sucrose (mg/gDW),2- insoluble CHO (mg/gDW),3- protein concentration (mg/gDW),4- amino acid (umol/gDW)
0,36.952343,19.731656,383.846043,222.3031


In [9]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

cobra/core/metabolite.py:103 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 25.9492553613 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 5.7497158698 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)


In [10]:
biomass.at["sSUCROSE_b","leaf"]=float(df_content_std_1["1- sucrose (mg/gDW)"][0])/model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["Starch_b","leaf"]=float(df_content_std_1["2- insoluble CHO (mg/gDW)"][0])/model.metabolites.get_by_id("STARCH_p").formula_weight
biomass.at["Protein_b","leaf"]=float(df_content_std_1["3- protein concentration (mg/gDW)"][0])/proteinMW
biomass.at["ARG_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("ARG_c").formula_weight)
biomass.at["HIS_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("HIS_c").formula_weight)
biomass.at["LYS_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("LYS_c").formula_weight)
biomass.at["sASP_b","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight)
biomass.at["sGLU_b","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("GLT_c").formula_weight)
biomass.at["sSER_b","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("SER_c").formula_weight)
biomass.at["THR_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("THR_c").formula_weight)
biomass.at["ASN_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("ASN_c").formula_weight)
biomass.at["sGLN_b","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("GLN_c").formula_weight)
biomass.at["CYS_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("CYS_c").formula_weight)
biomass.at["GLY_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("GLY_c").formula_weight)
biomass.at["PRO_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("PRO_c").formula_weight)
biomass.at["sALA_b","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight)
biomass.at["VAL_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("VAL_c").formula_weight)
biomass.at["ILE_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("ILE_c").formula_weight)
biomass.at["LEU_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("LEU_c").formula_weight)
biomass.at["MET_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("MET_c").formula_weight)
biomass.at["PHE_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("PHE_c").formula_weight)
biomass.at["TYR_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("TYR_c").formula_weight)
biomass.at["TRP_c","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("TRP_c").formula_weight)
biomass.at["sGABA_b","leaf"]=float(df_content_std_1["4- amino acid (umol/gDW)"][0])/(21.0*model.metabolites.get_by_id("4_AMINO_BUTYRATE_c").formula_weight)

### Stem

Extracting data from Shi et al 2016 Figure 3

<img src="references/Screenshot_20190711_154110.png">

In [11]:
df_stem_content_1 = pd.DataFrame(data={"DPA":[15,15,15,15,20,20,20,20,25,25,25,25,30,30,30,30],
                                       "Soluble sugar content (mg/gDW)":[378.125,424.21875,484.375,
                                                                         508.59375,295.3125,314.0625,
                                                                         369.53125,385.9375,104.6875,
                                                                         95.3125000000002,211.71875,
                                                                         203.125,115.625,64.0625000000001,
                                                                         64.8437500000002,46.0937500000002]})
df_stem_content_1

,DPA,Soluble sugar content (mg/gDW)
0,15,378.12500
1,15,424.21875
2,15,484.37500
3,15,508.59375
4,20,295.31250
5,20,314.06250
6,20,369.53125
7,20,385.93750
8,25,104.68750
9,25,95.31250


Extracting data from Scofield et al 2009 Figure 6

<img src="references/Screenshot_20190711_161803.png" height=50% width=50%>

In [12]:
df_stem_content_2 = pd.DataFrame(data={"DPA":[-6,2,18,28,35],
                                       "Starch (mg/gDW)":[0.47149917627677096,0.7037891268533771,
                                                          0.35782537067545306,0.03657331136738069,0],
                                       "soluble CHO (mg/gDW)":[70.71917808219148,141.60958904109575,
                                                               403.5958904109593,281.33561643835634,
                                                               375.856164383562]})
df_stem_content_2

,DPA,Starch (mg/gDW),soluble CHO (mg/gDW)
0,-6,0.471499,70.719178
1,2,0.703789,141.609589
2,18,0.357825,403.595890
3,28,0.036573,281.335616
4,35,0.000000,375.856164


Extracting lignin content from Zheng et al 2017, Figure 3

<img src="references/Screenshot_20190712_095308.png" height=50% width=50%>

In [13]:
df_stem_content_3 = pd.DataFrame(data={"DPA":[0,0,0,0,7,7,7,7,14,14,14,14,21,21,21,21,28,28,28,28,
                                              35,35,35,35,42,42,42,42,49,49,49,49],
                                       "lignin content OD280 /mL/gFW":[32.183908045977,42.5287356321839,27.2727272727272,
                                                         47.7272727272727,147.126436781609,165.51724137931,
                                                         138.636363636364,165.909090909091,293.103448275862,
                                                         316.091954022988,272.727272727273,306.818181818182,
                                                         352.873563218391,367.816091954023,339.772727272728,
                                                         365.909090909091,400,437.931034482758,400.000000000001,
                                                         434.09090909091,448.275862068965,480.459770114942,
                                                         447.727272727273,476.136363636364,475.862068965517,
                                                         493.103448275862,464.772727272728,493.181818181819,
                                                         480.459770114942,501.149425287356,473.863636363637,
                                                         501.136363636364]})

In [14]:
biomass.at["sSUCROSE_b","stem"] =  df_stem_content_2[df_stem_content_2["DPA"]==-6]["soluble CHO (mg/gDW)"]/ \
                                        (model.metabolites.get_by_id("SUCROSE_c").formula_weight)
biomass.at["Starch_b","stem"] = df_stem_content_2[df_stem_content_2["DPA"]==-6]["Starch (mg/gDW)"]/ \
                                        (model.metabolites.get_by_id("GLC_c").formula_weight)

,leaf,root,seed,stem
,,,,
sSUCROSE_b,0.107954,0.0,0.0,1.033279
GLC_c,0.000000,0.0,0.0,0.000000
FRU_c,0.000000,0.0,0.0,0.000000
Starch_b,0.121695,0.0,0.0,0.357825
Cellulose_b,0.000000,0.0,0.0,0.000000
PALMITATE_c,0.000000,0.0,0.0,0.000000
L_PHOSPHATIDATE_p,0.000000,0.0,0.0,0.000000
PHOSPHATIDYL_CHOLINE_r,0.000000,0.0,0.0,0.000000
L_1_PHOSPHATIDYL_ETHANOLAMINE_r,0.000000,0.0,0.0,0.000000


### Summary

- Only CHO data is available

Assumptions:

- 20DPA data from Shi et al 2016 were combined with 18DPA data from Scofield et al 2016

### Seed
Extracted sucrose and starch content data from Zi et al 2018, Figure 2

<img src="references/Screenshot_20190712_114010.png" height=50% width=50%>

In [15]:
df_seed_content_1 = pd.DataFrame(data={"Sucrose content (mg/gDW)":[17.9802955665025,22.7832512315271,
                                                                   9.92610837438423,8.66995073891626,
                                                                   34.0147783251232,42.7339901477833,
                                                                   10.7389162561576,15.4679802955665],
                                       "Starch content (mg/gDW)":[59.4789356984479,58.6363636363636,
                                                                  66.1751662971175,67.2838137472284,
                                                                  58.5476718403547,58.680709534368,
                                                                  64.8891352549889,65.3325942350332]})
df_seed_content_1

,Starch content (mg/gDW),Sucrose content (mg/gDW)
0,59.478936,17.980296
1,58.636364,22.783251
2,66.175166,9.926108
3,67.283814,8.669951
4,58.547672,34.014778
5,58.680710,42.733990
6,64.889135,10.738916
7,65.332594,15.467980


From Li et al 2016, get AA composition and use AA content from legend (15.90 mg/gDW) to get mg/gDW data


In [16]:
df_seed_content_2 = pd.DataFrame(data={"Amino acid":["Ala","Gly","Val","Leu","Ile","Pro","Phe",
                                                     "Met","Thr","Ser","Tyr","Cys","Arg","Asp",
                                                     "His","Glt","Lys"],
                                       "content mg/gC":[4.9,4.1,7.6,13.4,6.2,20.4,11.5,2.4,4.2,
                                                        6.2,3.8,2.2,6.1,5.4,3.5,52.3,4.1]})
df_seed_content_2["AA fraction"]=df_seed_content_2["content mg/gC"]/sum(df_seed_content_2["content mg/gC"])
df_seed_content_2["content mg/gDW"]=df_seed_content_2["AA fraction"]*15.90
df_seed_content_2

,Amino acid,content mg/gC,AA fraction,content mg/gDW
0,Ala,4.9,0.030954,0.492167
1,Gly,4.1,0.025900,0.411813
2,Val,7.6,0.048010,0.763361
3,Leu,13.4,0.084649,1.345925
4,Ile,6.2,0.039166,0.622742
5,Pro,20.4,0.128869,2.049021
6,Phe,11.5,0.072647,1.155085
7,Met,2.4,0.015161,0.241061
8,Thr,4.2,0.026532,0.421857
9,Ser,6.2,0.039166,0.622742


Protein amino acid composition was gathered from Jiang et al 2008 Table 2 and Table 5

In [17]:
df_seed_content_3 = pd.DataFrame(data={"AA":["LYS","THR","PHE","ILE","LEU","VAL","MET","ARG",
                                       "HIS","GLU","PRO","GLY","ALA","CYS","TYR","ASP","SER"],
                                       "AA composition (mean of 17 species) g/100g protein":[2.74,2.83,4.17,3.42,5.99,3.90,1.04,4.09,2.81,
                                                29.96,9.12,3.59,3.37,1.57,1.80,4.22,3.32]})
df_seed_content_3=df_seed_content_3.set_index("AA")
df_seed_content_3

,AA composition (mean of 17 species) g/100g protein
AA,
LYS,2.74
THR,2.83
PHE,4.17
ILE,3.42
LEU,5.99
VAL,3.90
MET,1.04
ARG,4.09
HIS,2.81


Extract total phospholipid composition from Gonzalez-Thuillier et al 2015, Figure 2

<img src="references/Screenshot_20190712_153403.png" height=50% width=50%>

In [18]:
df_seed_content_4 = pd.DataFrame(data={"Sample":["B1","B2","B3","R1","R2","R3","OF","O-OT","BF","B-OT"],
                                       "free FA":[34.73684210526337,32.14285714285759,
                                                  30.000000000000412,23.39999999999948,
                                                  17.99999999999964,_,
                                                  _,_,
                                                  _,_],
                                       "DAG":[4.818181818181818,4.545454545454586,
                                              6.463414634146272,4.93243243243235,
                                              5.714285714285113,_,
                                              _,_,
                                              _,_],
                                       "TAG":[10.272727272727273,14.204545454545604,
                                              36.4285714285718,20.999999999999524,
                                              49.03846153846062,_,
                                              _,_,
                                              _,_],
                                       "LPC":[26.315789473684525,26.785714285714832,
                                              13.170731707316978,21.59999999999953,
                                              12.428571428571002,_,
                                              _,_,
                                              _,_],
                                       "PC":[2.2727272727272734,3.2954545454545876,
                                             1.4634146341462713,4.999999999999897,
                                             1.8571428571421884,_,
                                             _,_,
                                             _,_],
                                       "MGDG":[4.181818181818185,2.159090909090933,
                                               0.9756097560974962,3.986486486486372,
                                               1.7142857142850556,_,
                                               _,_,
                                               _,_],
                                       "DGDG":[12,12.954545454545594,
                                               7.073170731707243,13.799999999999596,
                                               6.285714285713674,_,
                                               _,_,
                                               _,_],
                                       "Minor PL":[3.545454545454543,3.522727272727323,
                                                   3.170731707317004,3.5810810810809897,
                                                   2.8571428571422413,_,
                                                   _,_,
                                                   _,_]})

In [19]:
df_seed_content_2

,Amino acid,content mg/gC,AA fraction,content mg/gDW
0,Ala,4.9,0.030954,0.492167
1,Gly,4.1,0.025900,0.411813
2,Val,7.6,0.048010,0.763361
3,Leu,13.4,0.084649,1.345925
4,Ile,6.2,0.039166,0.622742
5,Pro,20.4,0.128869,2.049021
6,Phe,11.5,0.072647,1.155085
7,Met,2.4,0.015161,0.241061
8,Thr,4.2,0.026532,0.421857
9,Ser,6.2,0.039166,0.622742


In [29]:
biomass.at["sSUCROSE_b","seed"] = find_average(df_seed_content_1["Sucrose content (mg/gDW)"])/ \
                                        (model.metabolites.get_by_id("SUCROSE_c").formula_weight)
biomass.at["Starch_b","seed"] = find_average(df_seed_content_1["Starch content (mg/gDW)"])/ \
                                        (model.metabolites.get_by_id("GLC_c").formula_weight)
biomass.at["ARG_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Arg"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("ARG_c").formula_weight)
biomass.at["HIS_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="His"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("HIS_c").formula_weight)
biomass.at["LYS_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Lys"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("LYS_c").formula_weight)
biomass.at["sASP_b","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Asp"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("L_ASPARTATE_c").formula_weight)
biomass.at["sSER_b","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Ser"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("SER_c").formula_weight)
biomass.at["THR_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Thr"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("THR_c").formula_weight)
biomass.at["CYS_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Cys"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("CYS_c").formula_weight)
biomass.at["GLY_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Gly"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("GLY_c").formula_weight)
biomass.at["PRO_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Pro"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("PRO_c").formula_weight)
biomass.at["sALA_b","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Ala"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("L_ALPHA_ALANINE_c").formula_weight)
biomass.at["VAL_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Val"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("VAL_c").formula_weight)
biomass.at["ILE_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Ile"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("ILE_c").formula_weight)
biomass.at["LEU_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Leu"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("LEU_c").formula_weight)
biomass.at["MET_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Met"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("MET_c").formula_weight)
biomass.at["PHE_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Phe"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("PHE_c").formula_weight)
biomass.at["TYR_c","seed"] = float(df_seed_content_2[df_seed_content_2["Amino acid"]=="Tyr"]["content mg/gDW"])/ \
                                        (model.metabolites.get_by_id("TYR_c").formula_weight)
biomass.to_csv("/home/sanu/biomass_wheat.csv")

### Mature leaves

Extract day-time starch:sucrose accumulation rate ratio from Trvanion 2000, Figure 3

<img src="references/Screenshot_20190713_103920.png" height=50% width=50%>

In [21]:
df_matureleaf_content_1 = pd.DataFrame(data={"Time":["Dusk 1","Dawn 1","Dusk 2","Dawn 2"],
                                             "Sucrose (umol hexose/mgChl)":[33.54771784232365,
                                                                            2.3236514522821494,
                                                                            32.71784232365145,
                                                                            2.1161825726141004],
                                             "Starch (umol hexose/mgChl)":[10.878205128205131,
                                                                           1.23076923076923,
                                                                           10.814102564102562,
                                                                           1.4871794871794926]})

In [22]:
Delta_sucrose_day1 = float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk 1"]) - \
float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn 1"])

Delta_sucrose_day2 =float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk 2"]) - \
float(df_matureleaf_content_1["Sucrose (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn 2"])

Delta_starch_day1 = float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk 1"]) - \
float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn 1"])

Delta_starch_day2 =float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dusk 2"]) - \
float(df_matureleaf_content_1["Starch (umol hexose/mgChl)"][df_matureleaf_content_1["Time"]=="Dawn 2"])

df_matureleaf_content_1["Starch (dusk-dawn)"]=["-",Delta_starch_day1,"-",Delta_starch_day2]
df_matureleaf_content_1["Sucrose (dusk-dawn)"]=["-",Delta_sucrose_day1,"-",Delta_sucrose_day2]
df_matureleaf_content_1

,Starch (umol hexose/mgChl),Sucrose (umol hexose/mgChl),Time,Starch (dusk-dawn),Sucrose (dusk-dawn)
0,10.878205,33.547718,Dusk 1,-,-
1,1.230769,2.323651,Dawn 1,9.64744,31.2241
2,10.814103,32.717842,Dusk 2,-,-
3,1.487179,2.116183,Dawn 2,9.32692,30.6017


In [24]:
Sucrose_in_umols = 0.5*(df_matureleaf_content_1.at[1,"Sucrose (dusk-dawn)"]+df_matureleaf_content_1.at[3,"Sucrose (dusk-dawn)"])/2
Starch_in_umols = (df_matureleaf_content_1.at[1,"Starch (dusk-dawn)"]+df_matureleaf_content_1.at[3,"Starch (dusk-dawn)"])/2
Starch_to_sucrose_ratio = Starch_in_umols/Sucrose_in_umols
print ("Starch sucrose day-time accumulation rate = "+str(Starch_to_sucrose_ratio))

Starch sucrose day-time accumulation rate = 0.613801411117


Find % increase in starch and sucrose content in 2 hrs into daytime

In [27]:
df_matureleaf_content_temp = pd.DataFrame(data={"Day":[1,2],
                                                "Δsucrose (% of dawn level)":
                                                [100*(3.3333333333332646-2.499999999999943)/2.499999999999943,
                                                 100*(5.416666666666522-2.0833333333331936)/2.0833333333331936],
                                                "Δstarch (% of dawn level)":
                                                [100*(1.2258064516128684-1.1612903225806193)/1.1612903225806193,
                                                 100*(1.1935483870967154-1.4193548387096264)/1.4193548387096264]})
df_matureleaf_content_temp

,Day,Δstarch (% of dawn level),Δsucrose (% of dawn level)
0,1,5.555556,33.333333
1,2,-15.909091,160.000000


### References

- Allwood, J. W., Chandra, S., Xu, Y., Dunn, W. B., Correa, E., Hopkins, L., … Bowsher, C. G. (2015). Profiling of spatial metabolite distributions in wheat leaves under normal and nitrate limiting conditions. Phytochemistry, 115(1), 99–111. https://doi.org/10.1016/j.phytochem.2015.01.007
- Shi, H., Wang, B., Yang, P., Li, Y., & Miao, F. (2016). Differences in Sugar Accumulation and Mobilization between Sequential and Non-Sequential Senescence Wheat Cultivars under Natural and Drought Conditions. PLOS ONE, 11(11), e0166155. https://doi.org/10.1371/journal.pone.0166155
- Hu, Y., Burucs, Z., & Schmidhalter, U. (2006). Short-term effect of drought and salinity on growth and mineral elements in wheat seedlings. Journal of Plant Nutrition, 29(12), 2227–2243. https://doi.org/10.1080/01904160600975111
- Scofield, G. N., Ruuska, S. A., Aoki, N., Lewis, D. C., Tabe, L. M., & Jenkins, C. L. D. (2009). Starch storage in the stems of wheat plants: localization and temporal changes. Annals of Botany, 103(6), 859–868. https://doi.org/10.1093/aob/mcp010
- Zheng, M., Chen, J., Shi, Y., Li, Y., Yin, Y., Yang, D., … Li, Y. (2017). Manipulation of lignin metabolism by plant densities and its relationship with lodging resistance in wheat. Scientific Reports, 7(1), 41805. https://doi.org/10.1038/srep41805
- Zi, Y., Ding, J., Song, J., Humphreys, G., Peng, Y., Li, C., … Guo, W. (2018). Grain Yield, Starch Content and Activities of Key Enzymes of Waxy and Non-waxy Wheat (Triticum aestivum L.). Scientific Reports, 8(1), 4548. https://doi.org/10.1038/s41598-018-22587-0
- Li, X., Jiang, D., & Liu, F. (2016). Dynamics of amino acid carbon and nitrogen and relationship with grain protein in wheat under elevated CO 2 and soil warming. Environmental and Experimental Botany, 132, 121–129. https://doi.org/10.1016/j.envexpbot.2016.08.013
- JIANG, X., TIAN, J., HAO, Z., & ZHANG, W. (2008). Protein Content and Amino Acid Composition in Grains of Wheat-Related Species. Agricultural Sciences in China, 7(3), 272–279. https://doi.org/10.1016/S1671-2927(08)60066-8
- González-Thuillier, I., Salt, L., Chope, G., Penson, S., Skeggs, P., Tosi, P., … Haslam, R. P. (2015). Distribution of Lipids in the Grain of Wheat (cv. Hereward) Determined by Lipidomic Analysis of Milling and Pearling Fractions. Journal of Agricultural and Food Chemistry, 63(49), 10705–10716. https://doi.org/10.1021/acs.jafc.5b05289
- Trevanion, S. J. (2000). Photosynthetic carbohydrate metabolism in wheat (Triticum aestivum L.) leaves: optimization of methods for determination of fructose 2,6-bisphosphate. Journal of Experimental Botany, 51(347), 1037–1045. https://doi.org/10.1093/jexbot/51.347.1037